In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Creating whl files for external packges is described on [HERE](https://www.kaggle.com/jiny333/creating-whl-files-to-install-external-libraries).

In [ ]:
# creating whl files: https://stackoverflow.com/questions/57942120/how-to-package-all-my-library-dependencies-in-wheel-file
!ls ../input/pycaret-231-whl/

In [ ]:
!unzip ../input/pycaret-231-whl/pycaret_2.3.1.zip -d ../pycaret

In [ ]:
!ls ../pycaret/

In [ ]:
# installing multiple whl files: https://stackoverflow.com/questions/65844775/how-to-install-multiple-whl-files-in-the-right-order
# this command installs all whl files in the wheelhouse folder
!pip install --no-index --no-deps ../pycaret/wheelhouse/*.whl

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [ ]:
!pip show pycaret

In [ ]:
from pycaret.regression import * 

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
submission = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
vec = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1, 3), min_df=50)
X_train = vec.fit_transform(train['excerpt'])
X_test = vec.transform(test['excerpt'])
print(X_train.shape, X_test.shape)

In [ ]:
train = pd.concat([pd.DataFrame(X_train.todense()), train.target], axis=1)

In [ ]:
X_test = pd.DataFrame(X_test.todense())

In [ ]:
reg = setup(train, target='target', fold=5, fold_shuffle=True, session_id=42, silent=True)

In [ ]:
top3 = compare_models(n_select=3)

In [ ]:
blender_specific = blend_models(estimator_list=top3)

In [ ]:
pred_holdout = predict_model(blender_specific)

In [ ]:
final = finalize_model(blender_specific)

In [ ]:
predictions = predict_model(final, data=X_test)

In [ ]:
submission['target'] = predictions['Label']
submission

In [ ]:
submission.to_csv('submission.csv', index=False)